In [21]:
import pygrib
import numpy as np
from pprint import pprint
import xarray
import time
import os


# ERA_interim数据集(/12h, lon:1 lat:0.5)
# 1-8月测试

In [22]:
filename = '/mnt/pami14/DATASET/METEOROLOGY/benchmark_ERA_interim/ec19_Aug.grib'

In [30]:
myfile = pygrib.open(filename)
print (type(myfile)) 
line = myfile.readline()
strline = str(line)

print (strline)
type (strline)

sp_strline = strline.split(":", 6)
print (sp_strline)

feature_name = sp_strline[1]
feature_level = sp_strline[5]

feature_info = feature_name + ' ' + feature_level 

<class 'pygrib.open'>
1:Sea-ice cover:(0 - 1) (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201908010000
['1', 'Sea-ice cover', '(0 - 1) (instant)', 'regular_ll', 'surface', 'level 0', 'fcst time 0 hrs:from 201908010000']


In [42]:
'''可以取出时间戳，可以在这里写循环做filter修改myfile，然后再重新组成myfile进行select，
问题是如何重组为myfile类型  ❌ 不成立没法修改和new
'''

'''可以用.select直接输入时间戳选择，但具体参数怎么设置时间戳，得看GitHub源码 ❌ 源码C写的，超级不好找
'''

'''
🤙
***可以直接在for data in myfile里面写判定，判定one_grib_message_1.analDate==‘指定时间戳’，然后取出
一波，在根据指定变量one_grib_message_1.name=='指定气象变量'进行filter,

***直接用.data截取lat/lon即可

其他操作都和之前pre_grib一样，按月操作保存(只和dh判定有关)，或者多月一齐操作分别按月保存(和m/d/h判定有关
且不用.data(lat,lon)做crop，方便！！！)，
都么的问题！！！

注意论文是取多层还是一层:  注意level
level 0 或 level None 的筛选
🤙

'''

output_path = '/home/yqliu/EC-BiasCorrecting/outputs/ERA5_interim'
# data_descript_l = []
# for data in myfile:
#     data_descript_l.append(data)
# #     print (type(data))


one_grib_message_1 = data_descript_l[1100]

one_grib_message_2 = data_descript_l[110]

name_test1_1 = one_grib_message_1.name
shortName_test1_1 = one_grib_message_1.shortName


print ('name_test1_1:', name_test1_1)
print ('shortName_test1_1:', shortName_test1_1)
print ('paramId_test1_1:', paramId_test1_1)
print ('name_test1_1_type:', type(name_test1_1))
print ('*'*20)
print (one_grib_message_1)

print ('*'*20)

time_test1_1 = one_grib_message_1.fcstimeunits
time_test2_1 = one_grib_message_1.analDate
time_test3_1 = one_grib_message_1.validDate
##
time_test1_2 = one_grib_message_2.fcstimeunits
time_test2_2 = one_grib_message_2.analDate
time_test3_2 = one_grib_message_2.validDate
##

print ('fcstimeunits:', time_test1_1)
print ('analDate:', time_test2_1)
print ('validDate:', time_test3_1)

print ('fcstimeunits:', type(time_test1_1))
print ('analDate:', type(time_test2_1))
print ('validDate:', type(time_test3_1))

print ('*'*20)

print ('fcstimeunits:', time_test1_2)
print ('analDate:', time_test2_2)
print ('validDate:', time_test3_2)


# print (data_descript_l[1])
    
# f = open(os.path.join(output_path, 'era5_meteo_vars_display.txt'), 'w')
# # # 必须是string类型才能保存，读取用eval()
# f.write(str(data_descript_l))
# f.close()


name_test1_1: Vertical integral of total energy
shortName_test1_1: vitoe
paramId_test1_1: 162063
name_test1_1_type: <class 'str'>
********************
1101:Vertical integral of total energy:J m**-2 (instant):regular_ll:entireAtmosphere:level 0:fcst time 0 hrs:from 201908041200
********************
fcstimeunits: hrs
analDate: 2019-08-04 12:00:00
validDate: 2019-08-04 12:00:00
fcstimeunits: <class 'str'>
analDate: <class 'datetime.datetime'>
validDate: <class 'datetime.datetime'>
********************
fcstimeunits: hrs
analDate: 2019-08-01 06:00:00
validDate: 2019-08-01 06:00:00


## 按数据文件和气象要素特征，来划分经纬度范围和时间

In [51]:
'''
一定不能只是按name来做.select去筛特征了，因为月、日都不同，
此数据是按月的，所以select后月相同，但日不同
其实自己写循环帅选会提高算法的时间效率，不用.select()，因为.select()要把里面的所有特征遍历一遍
注意层None和0的区别(如果没区别，就不要用.select())
'''

total_prep_levels = myfile.select(name = "Total precipitation")
total_prep = myfile.select(name = "Total precipitation")[0]

### level filter
total_prep_level0 =myfile.select(name = "Total precipitation", level=0)
'''
precipitation is not 'None'
'''
# total_prep_level_None =myfile.select(name = "Total precipitation", level=None)



print ('total_prep_levels:', total_prep_levels)
print (type(total_prep_levels))

print ('*'*20)

print ('total_prep:', total_prep)
print (type(total_prep))

print ('*'*20)

total_prep_data = total_prep.data(lat1 = 20 - 2, lat2 = 40 + 2, lon1 = 110 - 2, lon2 = 124 + 2)[0] * 1000
print ('total_prep_data:', total_prep_data)
print ('total_prep_data_type:', type(total_prep_data))
print ('total_prep_data_shape:', total_prep_data.shape)

print ('*'*20)

print ('total_prep_level0:', total_prep_level0)
print ('total_prep_level0_type:', type(total_prep_level0))

for i, level0_message in enumerate(total_prep_level0):
    print ('level0_message_type:', level0_message)
    if i==0:
        print ('level0_message_type:', type(level0_message))
    print ('--'*8)
    
    
print ('*'*20)
   
# print ('total_prep_level_None:', total_prep_level_None)
# print ('total_prep_level_None_type:', type(total_prep_level_None))

print ('spatial_resolution_x:', round((42-18+1)/total_prep_data.shape[1], 4))
print ('spatial_resolution_y:', round((126-108+1)/total_prep_data.shape[0], 4))
    



# before_time = time.time()
# after_time = time.time()


'''
如果level0和level None有区别，可以先用select把level0的都筛查出来，然后再做迭代器循环进行筛选

selected_grbs=grbs.select(shorName='gh',typeOfLevel='isobaricInhPa',level=10)
'''



total_prep_levels: [9618:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 3 hrs:from 201908010000, 9700:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 6 hrs:from 201908010000, 9782:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201908010000, 9864:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 12 hrs:from 201908010000, 9946:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 3 hrs:from 201908011200, 10028:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 6 hrs:from 201908011200, 10110:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201908011200, 10192:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 12 hrs:from 201908011200, 10274:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 3 hrs:from 201908020000, 10356:Total precipitation:m (instant):regular_ll:surface:level 0:fcst time 6 hrs:fr

"\n如果level0和level None有区别，可以先用select把level0的都筛查出来，然后再做迭代器循环进行筛选\n\nselected_grbs=grbs.select(shorName='gh',typeOfLevel='isobaricInhPa',level=10)\n"